In [ ]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

# input files
shp_file = r"D:\Changthang_WS_GL\5_Changthang_Grrasslands_20_Big_84.shp"
ras_file = r"D:\Changthang_WS_GL\MODIS_GPP_Yearly_2001_2023.tif"

# read polygons
polygons = gpd.read_file(shp_file)

# open raster
ras = rasterio.open(ras_file)
n_bands = ras.count   # should be 23 bands (2001–2023)

# loop over bands and calculate mean for each polygon
for b in range(1, n_bands + 1):
    zs = zonal_stats(
        vectors=polygons,
        raster=ras_file,
        stats=["mean"],
        band=b,
        nodata=ras.nodata,
        geojson_out=True
    )
    
    # collect mean values
    vals = [f["properties"]["mean"] for f in zs]
    
    # band 1 = 2001, band 23 = 2023
    year = 2000 + b
    polygons[f"GPP_{year}"] = vals

# save output shapefile
out_file = r"D:\Changthang_WS_GL\Grasslands_GPP_ZonalStats.shp"
polygons.to_file(out_file)

print("Zonal stats done. Output saved:", out_file)

In [ ]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

# shapefile with polygons
shp_file = r"D:\Changthang_WS_GL\5_Changthang_Grrasslands_20_Big_84.shp"

# rasters to process: (path, prefix, start year)
rasters = [
    (r"D:\Changthang_WS_GL\MODIS_GPP_Yearly_2001_2023.tif", "GPP", 2001),
    (r"D:\Changthang_WS_GL\MODIS_NPP_Yearly_2001_2023.tif", "NPP", 2001),
    (r"D:\Changthang_WS_GL\Yearly_NDVI_Landsat7_2001_2013.tif", "NDVI", 2001),
    (r"D:\Changthang_WS_GL\Yearly_NDVI_Landsat8_2014_2023.tif", "NDVI", 2014),
]

# read polygons
gdf = gpd.read_file(shp_file)

def add_stats(gdf, raster_path, prefix, start_year):
    """add mean zonal stats from raster bands"""
    src = rasterio.open(raster_path)
    n_bands = src.count
    
    for b in range(1, n_bands + 1):
        zs = zonal_stats(
            vectors=gdf,
            raster=raster_path,
            stats=["mean"],
            band=b,
            nodata=src.nodata,
            geojson_out=True
        )
        vals = [f["properties"]["mean"] for f in zs]
        year = start_year + (b - 1)
        gdf[f"{prefix}_{year}"] = vals
    
    src.close()
    return gdf

# run for all rasters
for path, prefix, start_year in rasters:
    gdf = add_stats(gdf, path, prefix, start_year)

# save result
out_file = r"D:\Changthang_WS_GL\6_Changthang_Grrasslands_20_Zonal_Stat_84.shp"
gdf.to_file(out_file)

print("Zonal statistics finished. Output saved:", out_file)
